In [1]:
# Authors: Marijn van Vliet <w.m.vanvliet@gmail.com>
#          Alex Rockhill <aprockhill@mailbox.org>
#
# License: BSD-3-Clause

import numpy as np
import mne
from mne.datasets import somato
from mne.time_frequency import tfr_morlet, csd_tfr
from mne.beamformer import make_dics, apply_dics_tfr_epochs

print(__doc__)

Automatically created module for IPython interactive environment


In [2]:
data_path = somato.data_path()
subject = "01"
task = "somato"
raw_fname = data_path / f"sub-{subject}" / "meg" / f"sub-{subject}_task-{task}_meg.fif"
fname_fwd = (
    data_path / "derivatives" / f"sub-{subject}" / f"sub-{subject}_task-{task}-fwd.fif"
)
subjects_dir = data_path / "derivatives" / "freesurfer" / "subjects"

In [3]:
# Load raw data and make epochs.
raw = mne.io.read_raw_fif(raw_fname)
events = mne.find_events(raw)
epochs = mne.Epochs(
    raw,
    events,
    event_id=1,
    tmin=-1,
    tmax=2.5,
    reject=dict(
        grad=5000e-13,  # unit: T / m (gradiometers)
        mag=5e-12,  # unit: T (magnetometers)
        eog=250e-6,  # unit: V (EOG channels)
    ),
    preload=True,
)
epochs = epochs[:10]  # just for speed of execution for the tutorial

# We are mostly interested in the beta band since it has been shown to be
# active for somatosensory stimulation
freqs = np.linspace(13, 31, 5)

# Use Morlet wavelets to compute sensor-level time-frequency (TFR)
# decomposition for each epoch. We must pass ``output='complex'`` if we wish to
# use this TFR later with a DICS beamformer. We also pass ``average=False`` to
# compute the TFR for each individual epoch.
epochs_tfr = tfr_morlet(
    epochs, freqs, n_cycles=5, return_itc=False, output="complex", average=False
)

# crop either side to use a buffer to remove edge artifact
epochs_tfr.crop(tmin=-0.5, tmax=2)

Opening raw data file /home/paosoriom/mne_data/MNE-somato-data/sub-01/meg/sub-01_task-somato_meg.fif...
    Range : 237600 ... 506999 =    791.189 ...  1688.266 secs
Ready.
111 events found
Event IDs: [1]
Not setting metadata
111 matching events found
Setting baseline interval to [-0.9989760657919393, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Loading data for 111 events and 1052 original time points ...
    Rejecting  epoch based on MAG : ['MEG 0121']
    Rejecting  epoch based on MAG : ['MEG 0121']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on MAG : ['MEG 0211', 'MEG 1331', 'MEG 2211', 'MEG 2241', 'MEG 2521']
    Rejecting  epoch based on EOG : ['EOG 061']
    Rejecting  epoch based on MAG : ['MEG 1641', 'MEG 1831', 'MEG 1921', 'MEG 1941', 'MEG 2241']
    Rejecting  epoch based on MAG : ['MEG 1831']
    Rejecting  epoch based on MAG : ['MEG 1611']
    Rejecting  epoch based on MAG : ['MEG 0211', 'MEG 0441', 'MEG 1631'

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Not setting metadata


[Parallel(n_jobs=1)]: Done 306 out of 306 | elapsed:    1.8s finished


<EpochsTFR | time : [-0.499488, 2.001282], freq : [13.000000, 31.000000], epochs : 10, channels : 306, ~178.9 MB>

In [4]:
# Compute the Cross-Spectral Density (CSD) matrix for the sensor-level TFRs.
# We are interested in increases in power relative to the baseline period, so
# we will make a separate CSD for just that period as well.
csd = csd_tfr(epochs_tfr, tmin=-0.5, tmax=2)
baseline_csd = csd_tfr(epochs_tfr, tmin=-0.5, tmax=-0.1)

# use the CSDs and the forward model to build the DICS beamformer
fwd = mne.read_forward_solution(fname_fwd)

# compute scalar DICS beamfomer
filters = make_dics(
    epochs.info,
    fwd,
    csd,
    noise_csd=baseline_csd,
    pick_ori="max-power",
    reduce_rank=True,
    real_filter=True,
)

# project the TFR for each epoch to source space
epochs_stcs = apply_dics_tfr_epochs(epochs_tfr, filters, return_generator=True)

# average across frequencies and epochs
data = np.zeros((fwd["nsource"], epochs_tfr.times.size))
for epoch_stcs in epochs_stcs:
    for stc in epoch_stcs:
        data += (stc.data * np.conj(stc.data)).real

stc.data = data / len(epochs) / len(freqs)

# apply a baseline correction
stc.apply_baseline((-0.5, -0.1))

Reading forward solution from /home/paosoriom/mne_data/MNE-somato-data/derivatives/sub-01/sub-01_task-somato-fwd.fif...
    Reading a source space...
    [done]
    Reading a source space...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read MEG forward solution (8155 sources, 306 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame
Identifying common channels ...
Dropped the following channels:
['STI 001', 'STI 002', 'STI 016', 'STI 005', 'STI 015', 'STI 004', 'EOG 061', 'STI 014', 'STI 003', 'STI 006']
Identifying common channels ...
Computing inverse operator with 306 channels.
    306 out of 306 channels remain after picking
Selected 306 channels
Creating the depth weighting matrix...
Whitening the forward solution.
Computing rank from covariance with rank=None
    Using tolerance 2.8e-14 (2.2e-16 eps * 306 dim * 0.4  max singular value)
    Estimated rank (mag + grad): 64
    MEG: rank 64

<SourceEstimate | 8155 vertices, subject : 01, tmin : -499.48803289596964 (ms), tmax : 2001.2820518031856 (ms), tstep : 3.3299202193064645 (ms), data shape : (8155, 752), ~46.9 MB>

In [7]:
fmax = 4500
brain = stc.plot(
    subjects_dir=subjects_dir,
    hemi="both",
    views="dorsal",
    initial_time=0.55,
    brain_kwargs=dict(show=False),
    add_data_kwargs=dict(
        fmin=fmax / 10,
        fmid=fmax / 2,
        fmax=fmax,
        scale_factor=0.0001,
        colorbar_kwargs=dict(label_font_size=10),
    ),
)

# You can save a movie like the one on our documentation website with:
# brain.save_movie(tmin=0.55, tmax=1.5, interpolation='linear',
#                  time_viewer=True)

RuntimeError: Could not load any valid 3D backend
pyvistaqt: libGL.so.1: cannot open shared object file: No such file or directory
notebook: libGL.so.1: cannot open shared object file: No such file or directory

 install pyvistaqt, using pip or conda:
'pip install pyvistaqt'
'conda install -c conda-forge pyvistaqt'

 or install ipywidgets, if using a notebook backend
'pip install ipywidgets'
'conda install -c conda-forge ipywidgets'